# Clustering

In this notebook, we aim at creating a representation of all financial assets in the database that allows to group them according to some shared characteristics. This also includes dealing with categorical and null values as well as defining a preprocessing function.


### Import statements

In [1]:
# Import statements 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

### 1° Loading the dataset

In [2]:
filepath = '../../RFQ_Data_Challenge_HEC.csv'
df = pd.read_csv(filepath)
df.head()

C:\Users\gascm\AppData\Local\Temp\ipykernel_2544\1895652701.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


,Deal_Date,ISIN,cusip,Instrument,Sales_Name,Sales_Initial,company_short_name,B_Price,B_Side,Total_Requested_Volume,...,MidYTM,YTWDate,SpreadvsBenchmarkMid,MidASWSpread,MidZSpread,GSpreadMid,MidModifiedDuration,MidConvexity,MidEffectiveDuration,MidEffectiveConvexity
0,2022-09-15,DE000A14J587,D8397TBT2,TKAGR 25/2/2025 2.500,Blanca Tailpied,BLA,Bnp Reunion,0.0,NATIXIS SELL,77551,...,5.57551,NaN,417.014771,286.230682,295.597916,430.414978,2.23491,8.41587,2.11347,6.82874
1,2022-09-15,DE000A14J587,D8397TBT2,TKAGR 25/2/2025 2.500,Blanca Tailpied,BLA,Cegi Ett,0.0,NATIXIS SELL,8163,...,5.57551,NaN,417.014771,286.230682,295.597916,430.414978,2.23491,8.41587,2.11347,6.82874
2,2022-09-15,DE000A14J587,D8397TBT2,TKAGR 25/2/2025 2.500,Blanca Tailpied,BLA,Qbe,0.0,NATIXIS SELL,20408,...,5.57551,NaN,417.014771,286.230682,295.597916,430.414978,2.23491,8.41587,2.11347,6.82874
3,2022-09-15,DE000A14J587,D8397TBT2,TKAGR 2.5% 25 FEB 2025,Blanca Tailpied,BLA,Cegi Ett,0.0,NATIXIS SELL,16327,...,5.57551,NaN,417.014771,286.230682,295.597916,430.414978,2.23491,8.41587,2.11347,6.82874
4,2022-09-15,DE000A14J587,D8397TBT2,TKAGR 2.5% 25 FEB 2025,Constantin Durie,CON,Scp Laureau-Jeannerot,0.0,NATIXIS SELL,244898,...,5.57551,NaN,417.014771,286.230682,295.597916,430.414978,2.23491,8.41587,2.11347,6.82874


### 2° Defining the preprocessing function

Insights on some features :
- **Deal_Date** : The date on which a financial deal is executed. Needs to be converteted in datetime.
- **ISIN** : International Securities Identification Number, a unique identifier for Financial instruments.
- **company_short_name** : the name of the issuer of the financial instrument. It corresponds to the client name.
- **B_price** : The bid price of the financial instrument. To be converted to int (object for the moment)
- **B_side** : Natixis's position as a buyer or seller of the financial instrument (for the moment 'NATIXIS SELL' or 'NATIXIS BUY'). Contient 8 valeurs nulles.
- **Total_Requested_Volume** : The requested volume for buying or selling the financial instrument. It needs to be converted into a numerical column (object for the moment). Contient 2 valeurs nulles.
- **Total_Traded_Volume_Natixis** : The volume of the financial instrument traded by Natixis. Already good data type.
- **Total_Traded_Volume_Away** : The volume of the financial instrument traded by other banks. Already good data type.
- **Total_Traded_Volume** : The total volume of the financial instrument traded. Already good data type.
- **BloomIndustrySector**, **BloomIndustryGroup**,**BloomIndustrySubGroup**
- **maturity** : The length of time during which interest is paid. Some null values are marked as NaT. We convert this column into Datetime type. Some maturities go back to 1900, it is not possible. We delete those.
- **Rating_Fitch** : The credit rating of the financial instrument from Fitch Ratings.
- **Rating_Moodys** : The credit rating of the financial instrument from Moody's.
- **Rating_SP** : The credit rating of the financial instrument from S&P Global Ratings.
- **Ccy** : The currency in which the financial instrument is denominated.
- **Classification** : The activity sector of the company.
- **Tier** : The seniority level of the financial instrument. Lots of null value, we replace them by UNKOWN (627100 values)
- **AssumedMaturity** : The assumed maturity date of the financial instrument. Also contains a lot of null values, we replace them by maturity values (the null values only).
- **Coupon** : The interest rate of the financial instrument. Already a float.
- **Frequency** : The frequency of interest payments on the financial instrument. Takes values 1M, 3M, 6M, 12M. We delete the 'M' and convert the value into int.
- **Type** : The type of interest rate on the financial instrument (fixed or variable).
- **MidYTM** : The yield to maturity on the prime bid. Already a float.
- **MidYTM** : The yield to maturity on the prime bid. Already a float.
- **YTWDate** : Yield to Worst - The yield on the first possible redemption date. 
- **SpreadvsBenchmarkMid** : The spread of the financial instrument versus the interpolated government bond curve.
- **MidASWSpread** : The spread of the financial instrument versus the swap curve.
- **MidZSpread** : The spread of the financial instrument versus the zero- coupon curve.
- **GSpreadMid** : The spread of the financial instrument versus the interpolated government bond curve.
- **MidModifiedDuration** : The modified duration of the financial instrument. 
- **MidConvexity** : The convexity of the financial instrument.
- **MidEffectiveDuration** : The effective duration of the financial instrument.
- **MidEffectiveConvexity** : The effective convexity of the financial instrument.

Features that can be deleted at first : 
- **Cusip**, same as **cusip** but with more null values 
- **Maturity**, same as **maturity**

Added columns :
- Year, month, day of deal_date
- Year, month, day of maturity
- days to maturity

In [3]:
# Null values analysis for columns with null values below 15000
below_threshold = df.isnull().sum().sort_values(ascending=False) < 15000
print("Columns with null values below 15000:")
print(below_threshold[below_threshold].index)

# Null values analysis for columns with null values above 15000
above_threshold = df.isnull().sum().sort_values(ascending=False) >= 15000
print("\nColumns with null values above or equal to 15000:")
print(above_threshold[above_threshold].index)


Columns with null values below 15000:
Index(['Sales_Initial', 'Sales_Name', 'MidYTM', 'Coupon', 'Ccy',
       'BloomIndustryGroup', 'BloomIndustrySector', 'Instrument', 'cusip',
       'maturity', 'cdcissuerShortName', 'Frequency', 'MidPrice', 'cdcissuer',
       'company_short_name', 'BloomIndustrySubGroup', 'B_Price',
       'Classification', 'Total_Traded_Volume_Natixis', 'B_Side',
       'Total_Traded_Volume_Away', 'Total_Requested_Volume',
       'Total_Traded_Volume', 'Type', 'Maturity', 'ISIN', 'Deal_Date'],
      dtype='object')

Columns with null values above or equal to 15000:
Index(['Tier', 'AssumedMaturity', 'YTWDate', 'GSpreadMid',
       'SpreadvsBenchmarkMid', 'Cusip', 'MidEffectiveConvexity',
       'MidEffectiveDuration', 'MidASWSpread', 'Rating_Moodys', 'MidConvexity',
       'Rating_Fitch', 'MidZSpread', 'Rating_SP', 'MidModifiedDuration',
       'Country', 'lb_Platform_2'],
      dtype='object')


In [4]:
def preprocess_dataframe(df):
    """
    Preprocesses the input DataFrame with the following steps:
    1. Converts 'Deal_Date', 'maturity', 'AssumedMaturity', 'YTWDate' columns to datetime.
    2. Converts 'B_Side' column to boolean (1 for 'NATIXIS BUY', 0 for 'NATIXIS SELL').
    3. Converts 'B_Price' and 'Total_Requested_Volume' columns to integers.
    4. Fills null values in 'Tier', 'AssumedMaturity', and 'YTWDate' columns with 'UNKNOWN'.
    5. Converts 'Frequency' feature values into integers (removing 'M' from the end).
    6. Drops the unsused 'Cusip' column.

    Parameters:
    - df (DataFrame): Input DataFrame.

    Returns:
    - DataFrame: Processed DataFrame.
    """

    df = df.copy()

    # Drop null values only for columns below the threshold
    columns_to_delete_null_vales = ['MidYTM', 'Coupon', 'Ccy', 'cusip',
       'maturity', 'cdcissuerShortName', 'Frequency', 'MidPrice', 'cdcissuer',
       'company_short_name', 'BloomIndustrySubGroup', 'B_Price',
       'Total_Traded_Volume_Natixis', 'B_Side',
       'Total_Traded_Volume_Away', 'Total_Requested_Volume',
       'Total_Traded_Volume', 'Type', 'Maturity', 'ISIN', 'Deal_Date']
    df = df.dropna(subset=columns_to_delete_null_vales)

    # Convert 'B_Price', 'Total_Requested_Volume', 'Frequency' to integers
    df['Frequency'] = df['Frequency'].str.replace('M', '')
    numerical_columns = ['B_Price', 'Total_Requested_Volume', 'Frequency']
    df.dropna(subset=numerical_columns, inplace=True)
    for column in numerical_columns:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype(int)

    # Fix the error in the B_Price column
    df = df[df['B_Price'] >= 20]

    # Replace NaT with null values in the 'Maturity' column
    df['maturity'].replace({pd.NaT: np.nan}, inplace=True)

    # Convert 'Deal_Date', 'maturity', 'AssumedMaturity', 'YTWDate' to datetime
    df['Deal_Date'] = pd.to_datetime(df['Deal_Date'])
    df['maturity'] = pd.to_datetime(df['maturity'], errors='coerce',  format='%Y-%m-%d %H:%M:%S.%f')
    df['AssumedMaturity'] = pd.to_datetime(df['AssumedMaturity'], errors='coerce')
    df['YTWDate'] = pd.to_datetime(df['YTWDate'], errors='coerce')

    # Add year, month, day for clustering 
    df['Year_dealdate'] = df['Deal_Date'].dt.year
    df['Month_dealdate'] = df['Deal_Date'].dt.month
    df['Day_dealdate'] = df['Deal_Date'].dt.day
    df['Year_maturity'] = df['maturity'].dt.year
    df['Month_maturity'] = df['maturity'].dt.month
    df['Day_maturity'] = df['maturity'].dt.day

    # Delete maturities smaller than 2021 (as deal dates starts in 2021)
    df = df[df['maturity'].dt.year >= 2021]

    # Compute number of days between maturity and deal date
    df['Days_to_Maturity'] = (df['maturity'] - df['Deal_Date']).dt.days

    # Replace null values in 'AssumedMaturity' with values from 'Maturity'
    df['AssumedMaturity'] = df['AssumedMaturity'].fillna(df['Maturity'])

    # Convert 'B_Side' column to boolean (1 for 'NATIXIS BUY', 0 for 'NATIXIS SELL')
    df = df[df['B_Side'].isin(['NATIXIS SELL', 'NATIXIS BUY'])]
    df['B_Side'] = df['B_Side'].replace({'NATIXIS BUY': 1, 'NATIXIS SELL': 0})

    # Convert null values of 'Tier'
    df['Tier'].fillna('UNKNOWN', inplace=True)

    # Lower string names 
    df['Sales_Name'] = df['Sales_Name'].str.lower()
    df['company_short_name'] = df['company_short_name'].str.lower()

    # Drop unused columns
    columns_to_drop = ['Cusip', 'Maturity']
    df.drop(columns=columns_to_drop, inplace=True)

    return df

In [5]:
df_preprocessed = preprocess_dataframe(df)
pd.set_option('display.max_columns', None)
df_preprocessed.head()

,Deal_Date,ISIN,cusip,Instrument,Sales_Name,Sales_Initial,company_short_name,B_Price,B_Side,Total_Requested_Volume,Total_Traded_Volume_Natixis,Total_Traded_Volume_Away,Total_Traded_Volume,BloomIndustrySector,BloomIndustryGroup,BloomIndustrySubGroup,cdcissuerShortName,cdcissuer,Country,lb_Platform_2,maturity,Rating_Fitch,Rating_Moodys,Rating_SP,Ccy,Classification,Tier,AssumedMaturity,Coupon,Frequency,Type,MidPrice,MidYTM,YTWDate,SpreadvsBenchmarkMid,MidASWSpread,MidZSpread,GSpreadMid,MidModifiedDuration,MidConvexity,MidEffectiveDuration,MidEffectiveConvexity,Year_dealdate,Month_dealdate,Day_dealdate,Year_maturity,Month_maturity,Day_maturity,Days_to_Maturity
6,2022-11-10,NL0011220108,EK8199722,NETHER 0.25% 15 JUL 2025,heddy kerneves,HED,banque populaire de l'ouest,95,1,13061,0.0,13061.0,13061.0,Government,Sovereign,Sovereign,ROPAYSBAS,NETHERLANDS GOVERNMENT BOND,NETHERLANDS,EMEA,2025-07-15,AAA,Aaa,AAA,EUR,Government,UNKNOWN,2025-07-15,0.25,12,Fixed,95.125999,2.15064,NaT,NaN,-72.215584,-74.569401,NaN,2.60208,10.181840,2.47743,8.883440,2022,11,10,2025.0,7.0,15.0,978
7,2022-11-10,NL0011220108,EK8199722,NETHER 0.25% 15 JUL 2025,ian prechais,IAN,license de transport,95,0,136,0.0,136.0,136.0,Government,Sovereign,Sovereign,ROPAYSBAS,NETHERLANDS GOVERNMENT BOND,NETHERLANDS,EMEA,2025-07-15,AAA,Aaa,AAA,EUR,Government,UNKNOWN,2025-07-15,0.25,12,Fixed,95.125999,2.15064,NaT,NaN,-72.215584,-74.569401,NaN,2.60208,10.181840,2.47743,8.883440,2022,11,10,2025.0,7.0,15.0,978
8,2022-11-10,NL0011220108,EK8199722,NETHER 0.25% 15 JUL 2025,ian prechais,IAN,bp val de france,95,0,1224490,0.0,1224490.0,1224490.0,Government,Sovereign,Sovereign,ROPAYSBAS,NETHERLANDS GOVERNMENT BOND,NETHERLANDS,EMEA,2025-07-15,AAA,Aaa,AAA,EUR,Government,UNKNOWN,2025-07-15,0.25,12,Fixed,95.125999,2.15064,NaT,NaN,-72.215584,-74.569401,NaN,2.60208,10.181840,2.47743,8.883440,2022,11,10,2025.0,7.0,15.0,978
9,2022-11-10,NL0011220108,EK8199722,NETHER 0.25% 15 JUL 2025,elisene arslane,ELI,hsbc,95,0,91837,0.0,91837.0,91837.0,Government,Sovereign,Sovereign,ROPAYSBAS,NETHERLANDS GOVERNMENT BOND,NETHERLANDS,EMEA,2025-07-15,AAA,Aaa,AAA,EUR,Government,UNKNOWN,2025-07-15,0.25,12,Fixed,95.125999,2.15064,NaT,NaN,-72.215584,-74.569401,NaN,2.60208,10.181840,2.47743,8.883440,2022,11,10,2025.0,7.0,15.0,978
12,2023-10-18,IT0005530032,ZM3378175,BTPS 4.45% 01 SEP 2043,ian prechais,IAN,societe generale mer,92,0,2041,0.0,2041.0,2041.0,Government,Sovereign,Sovereign,REPITALIE,BTP,ITALY,EMEA,2043-09-01,BBB,Baa3,BBB,EUR,Government,UNKNOWN,2043-09-01,4.45,6,Fixed,90.022003,5.26552,NaT,213.429138,173.721893,183.643234,217.34491,12.58312,212.362656,11.40296,192.445251,2023,10,18,2043.0,9.0,1.0,7258


In [6]:
df_preprocessed.shape

(314718, 49)

In [7]:
# Function for imputing numerical missing values in the financial columns
def complete_nan_values(df):

    df_unique_isin = df.groupby('ISIN').first()
    columns = ['Classification', 'SpreadvsBenchmarkMid', 'MidASWSpread', 'MidZSpread', 'GSpreadMid', 
               'MidModifiedDuration', 'MidConvexity', 'MidEffectiveDuration', 'MidEffectiveConvexity', 'Year_dealdate', 'Month_dealdate']
    df_by_classification = df_unique_isin[columns].copy()
    df_by_classification = df_by_classification.groupby(['Classification', 'Year_dealdate']).mean().reset_index()

    df_group_by_industry = df_by_classification.groupby('Classification').mean().reset_index()
    numeric_columns = ['SpreadvsBenchmarkMid', 'MidASWSpread', 'MidZSpread', 'GSpreadMid', 
                       'MidModifiedDuration', 'MidConvexity', 'MidEffectiveDuration', 'MidEffectiveConvexity']
    
    df_by_classification['additional_column'] = df_by_classification['Classification'].astype(str) + ' - ' + df_by_classification['Year_dealdate'].astype(str)
    df['additional_column'] = df['Classification'].astype(str) + ' - ' + df['Year_dealdate'].astype(str)

    for column in numeric_columns:
        df_by_classification[column] = df_by_classification[column].fillna(df_by_classification['Classification'].map(df_group_by_industry.set_index('Classification')[column]))

    for column in numeric_columns:
        df[column] = df[column].fillna(df['additional_column'].map(df_by_classification.set_index('additional_column')[column]))

    df.drop(columns=['additional_column'], inplace=True)
    
    return df

In [8]:
df_filled = complete_nan_values(df_preprocessed)
missing_values = df_filled.isnull().sum()
missing_values[missing_values!=0]

Instrument                  788
Sales_Name                  888
Sales_Initial               888
BloomIndustrySector          33
BloomIndustryGroup           33
Country                    7555
lb_Platform_2              7555
Rating_Fitch               4462
Rating_Moodys              3233
Rating_SP                  4473
Classification                2
YTWDate                  308252
MidASWSpread                  5
MidZSpread                    5
GSpreadMid                    5
MidModifiedDuration           5
MidConvexity                  5
MidEffectiveDuration          5
MidEffectiveConvexity         5
dtype: int64

Once we've corrected the B_price error, which sometimes corresponds to yield values (by setting a minimum value of 20), only 314718 lines remain.

### 4° Defining the preprocessing function for clustering

In [9]:
def preprocess_clustering(df, cols_to_exclude):

    # Drop the columns that we exclude
    df = df.drop(cols_to_exclude, axis=1, errors='ignore')

    # Identify numerical columns
    numerical_columns = df.select_dtypes(include=['number']).columns

    # Transform 'Ccy' to 'is_euro' boolean column
    df['is_euro'] = (df['Ccy'] == 'EUR').astype(int)
    # Transform 'Type' to 'is_fixed' boolean column
    df['is_fixed'] = (df['Type'] == 'Fixed').astype(int)
    # Drop the original 'Ccy' and 'Type' columns
    df = df.drop(['Ccy', 'Type'], axis=1, errors='ignore')

    # Ordinal encoding for 'Rating_Fitch'
    rating_mapping = {
        'AAA': 22,
        'AA+': 21,
        'AA': 20,
        'AA-': 19,
        'A+': 18,
        'A': 17,
        'A-': 16,
        'BBB+': 15,
        'BBB': 14,
        'BBB-': 13,
        'BB+': 12,
        'BB': 11,
        'BB-': 10,
        'B+': 9,
        'B': 8,
        'B-': 7,
        'CCC+': 6,
        'CCC': 5,
        'CCC-': 4,
        'CC': 3,
        'C': 2,
        'WD': 1,
        'D': 0,
        'NR': np.nan
    }

    rating_mapping_moodys = {
        'Aaa': 22,
        'Aa1': 21,
        'Aa2': 20,
        '(P)Aa2': 20,
        'Aa3': 19,
        '(P)Aa3': 19,
        'A1': 18,
        '(P)A1': 18,
        'A2': 17,
        '(P)A2': 17,
        'A3': 16,
        '(P)A3': 16,
        'Baa1': 15,
        '(P)Baa1': 15,
        'Baa2': 14,
        '(P)Baa2': 14,
        'Baa3': 13,
        'Ba1': 12,
        'Ba2': 11,
        'Ba3': 10,
        'B1': 9,
        'B2': 8,
        'B3': 7,
        'Caa1': 6,
        'Caa2': 5,
        'Caa3': 4,
        'Ca': 2.5,
        'C': 0
    }

    df['Rating_Fitch_encoded'] = df['Rating_Fitch'].map(rating_mapping)
    df['Rating_SP_encoded'] = df['Rating_SP'].map(rating_mapping)
    df['Rating_Moodys_encoded'] = df['Rating_Moodys'].map(rating_mapping_moodys)
    # Create a unique Rating that averages the 3 Ratings and ignores missing values
    df['Rating'] = df[['Rating_Fitch_encoded', 'Rating_SP_encoded', 'Rating_Moodys_encoded']].mean(axis=1)

    # Map values in 'Country' column
    valid_countries = ['FRANCE', 'ITALY', 'GERMANY', 'NETHERLANDS', 'SPAIN']
    df['Country'] = df['Country'].apply(lambda x: x if x in valid_countries else 'OTHER')
    # Perform one-hot encoding
    df = pd.get_dummies(df, columns=['Country'], prefix='is')

    # Map values in 'Classification' column
    valid_classes = ['Financials', 'Government', 'Industrials', 'Utilities']
    df['Classification'] = df['Classification'].apply(lambda x: x if x in valid_classes else 'OTHER')
    # Perform one-hot encoding
    df = pd.get_dummies(df, columns=['Classification'], prefix='is')

    # Add newly created boolean columns and 'Rating' to agg_dict with average
    agg_dict = {col: 'mean' for col in ['is_euro', 'is_fixed', 'Rating']}
    agg_dict.update({col: 'first' for col in ['is_FRANCE', 'is_ITALY', 'is_GERMANY', 'is_NETHERLANDS', 'is_SPAIN']})
    agg_dict.update({col: 'first' for col in ['is_Financials', 'is_Government', 'is_Industrials', 'is_Utilities']})
    agg_dict.update({num_col: ['min', 'max', 'median'] for num_col in numerical_columns})

    # Grouping by 'ISIN' and aggregating columns
    grouped_df = df.groupby('ISIN').agg(agg_dict).reset_index()

    # Flatten the multi-level column index
    grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]

    # Drop identical columns
    grouped_df = grouped_df.T.drop_duplicates().T

    return grouped_df

In [10]:
cols_to_exclude = ['Deal_Date', 'cusip', 'B_Side', 'Instrument', 'Sales_Name', 'Sales_Initial', 'company_short_name',
                   'Total_Requested_Volume', 'Total_Traded_Volume_Natixis', 'Total_Traded_Volume_Away', 'Total_Traded_Volume',
                   'cdissuer', 'Tier', 'Year_dealdate', 'Month_dealdate','Day_dealdate', 'Days_to_Maturity',
                   'cdissuerShortName', 'lb_Platform_2']
df_clustering = preprocess_clustering(df_filled, cols_to_exclude)

In [11]:
df_clustering.head()

,ISIN_,is_euro_mean,is_fixed_mean,Rating_mean,is_FRANCE_first,is_ITALY_first,is_GERMANY_first,is_NETHERLANDS_first,is_SPAIN_first,is_Financials_first,is_Government_first,is_Industrials_first,is_Utilities_first,B_Price_min,B_Price_max,B_Price_median,Coupon_min,Coupon_max,Coupon_median,Frequency_min,Frequency_max,MidPrice_min,MidPrice_max,MidPrice_median,MidYTM_min,MidYTM_max,MidYTM_median,SpreadvsBenchmarkMid_min,SpreadvsBenchmarkMid_max,SpreadvsBenchmarkMid_median,MidASWSpread_min,MidASWSpread_max,MidASWSpread_median,MidZSpread_min,MidZSpread_max,MidZSpread_median,GSpreadMid_min,GSpreadMid_max,GSpreadMid_median,MidModifiedDuration_min,MidModifiedDuration_max,MidModifiedDuration_median,MidConvexity_min,MidConvexity_max,MidConvexity_median,MidEffectiveDuration_min,MidEffectiveDuration_max,MidEffectiveDuration_median,MidEffectiveConvexity_min,MidEffectiveConvexity_max,MidEffectiveConvexity_median,Year_maturity_min,Month_maturity_min,Day_maturity_min
0,AT0000383864,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,110,126,112.0,6.25,6.25,6.25,12,12,110.419762,127.079163,112.316982,0.6494,3.23769,2.94855,14.74143,48.205238,31.070089,-92.459953,-23.11165,-36.86031,-80.192131,-21.09169,-34.07125,7.43871,51.626171,35.860584,3.16226,4.41606,3.498495,13.70098,25.4069,16.758875,3.25192,5.63155,4.03505,11.2101,32.001419,19.12505,2027.0,7.0,15.0
1,AT0000A04967,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,99,131,111.0,4.15,4.15,4.15,12,12,103.552841,132.494583,110.889221,1.63245,3.80473,3.166545,50.77877,75.725662,62.114321,-19.249849,30.029989,14.9261,-17.64529,27.424015,13.593732,40.751419,85.924889,64.691807,9.97172,11.42595,10.46476,126.879807,166.019135,140.971497,10.33749,15.32064,11.76933,125.579231,216.827042,155.175697,2037.0,3.0,15.0
2,AT0000A0DXC2,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,103,113,104.0,4.85,4.85,4.85,12,12,103.538582,113.266792,104.862549,1.0297,3.3129,2.95505,-3.98206,61.020901,50.342121,-88.60849,-30.5749,-55.127319,-82.060589,-29.441706,-52.924715,-7.01188,40.519329,19.612419,2.0336,3.27347,2.50163,6.31344,15.90359,9.58242,2.10278,3.77821,2.65142,4.62418,16.69602,9.41744,2026.0,3.0,15.0
3,AT0000A0U299,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,101,153,117.0,3.8,3.8,3.8,12,12,99.669472,154.466949,116.114517,1.84513,3.81589,3.08127,54.143429,85.863159,60.618561,57.923691,100.983322,75.536491,42.798744,82.300621,64.11986,60.884682,100.101662,73.465332,19.42235,23.934839,21.209641,569.411377,793.869263,656.415588,19.908159,37.439911,24.87862,583.653931,1222.869385,760.366577,2062.0,1.0,26.0
4,AT0000A0U3T4,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,100,100,100.0,3.4,3.4,3.4,12,12,100.032471,101.040138,100.26965,-0.2633,0.99577,0.75603,-92.022652,11.07714,-54.067989,-6.115799,-6.115799,-6.115799,-155.807577,-62.955996,-125.770611,-75.230278,27.381149,-13.132235,0.01356,0.30947,0.10065,0.02686,0.6187,0.199865,0.01416,0.32004,0.104665,0.00039,0.19814,0.021345,2022.0,11.0,22.0


### 5° Supervised clustering

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df_clustering.set_index('ISIN_', inplace=True)

Let's now deep dive into the classical KMeans where we will be imputing the missing values. Financial missing values have been imputed previously. We just need to impute the ratings. We will proceed with the median.

Imputing missing values:

In [ ]:
missing_values = df_clustering.isnull().sum()
missing_values[missing_values!=0] 

In [ ]:
df_clustering_filled = df_clustering.copy()
df_clustering_filled['Rating_mean'] = df_clustering_filled['Rating_mean'].fillna(df_clustering['Rating_mean'].median())

In [ ]:
scaler = StandardScaler()
df_normalized = scaler.fit_transform(df_clustering_filled)

We apply the elbow method to determine the optimal number of clusters for the KMeans approach.

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from umap.umap_ import UMAP

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(4,12))

In [ ]:
visualizer.fit(df_normalized)        # Fit the data to the visualizer
visualizer.show();                          # Finalize and render the figure

Optimal number of cluster is 7.
<br>
Now let's move on to exploring the results we obtain.

In [ ]:
model = KMeans(n_clusters=7, verbose=0, random_state=42)

In [ ]:
clusters = model.fit_predict(df_normalized)

In [ ]:
pd.Series(clusters).value_counts()

In [ ]:
embedding = UMAP(n_neighbors=50, learning_rate=0.5, init="random", min_dist=0.001
                      ).fit_transform(df_normalized)

In [ ]:
sns.scatterplot(x=embedding[:,0], y=embedding[:,1], hue=clusters, palette='dark')

Explainability options:
- ExKMC
- Build a classification model for each label and look at Shap values

### Building an explainability classification model

In [ ]:
from catboost import CatBoostClassifier
import shap

In [ ]:
def preprocess_explainability(df, cols_to_exclude):

    # Drop the columns that we exclude
    df = df.drop(cols_to_exclude, axis=1, errors='ignore')

    # Identify numerical columns
    numerical_columns = df.select_dtypes(include=['number']).columns

    # Transform 'Ccy' to 'is_euro' boolean column
    df['is_euro'] = (df['Ccy'] == 'EUR').astype(int)
    # Transform 'Type' to 'is_fixed' boolean column
    df['is_fixed'] = (df['Type'] == 'Fixed').astype(int)
    # Drop the original 'Ccy' and 'Type' columns
    df = df.drop(['Ccy', 'Type'], axis=1, errors='ignore')

    # Ordinal encoding for 'Rating_Fitch'
    rating_mapping = {
        'AAA': 22,
        'AA+': 21,
        'AA': 20,
        'AA-': 19,
        'A+': 18,
        'A': 17,
        'A-': 16,
        'BBB+': 15,
        'BBB': 14,
        'BBB-': 13,
        'BB+': 12,
        'BB': 11,
        'BB-': 10,
        'B+': 9,
        'B': 8,
        'B-': 7,
        'CCC+': 6,
        'CCC': 5,
        'CCC-': 4,
        'CC': 3,
        'C': 2,
        'WD': 1,
        'D': 0,
        'NR': np.nan
    }

    rating_mapping_moodys = {
        'Aaa': 22,
        'Aa1': 21,
        'Aa2': 20,
        '(P)Aa2': 20,
        'Aa3': 19,
        '(P)Aa3': 19,
        'A1': 18,
        '(P)A1': 18,
        'A2': 17,
        '(P)A2': 17,
        'A3': 16,
        '(P)A3': 16,
        'Baa1': 15,
        '(P)Baa1': 15,
        'Baa2': 14,
        '(P)Baa2': 14,
        'Baa3': 13,
        'Ba1': 12,
        'Ba2': 11,
        'Ba3': 10,
        'B1': 9,
        'B2': 8,
        'B3': 7,
        'Caa1': 6,
        'Caa2': 5,
        'Caa3': 4,
        'Ca': 2.5,
        'C': 0
    }

    df['Rating_Fitch_encoded'] = df['Rating_Fitch'].map(rating_mapping)
    df['Rating_SP_encoded'] = df['Rating_SP'].map(rating_mapping)
    df['Rating_Moodys_encoded'] = df['Rating_Moodys'].map(rating_mapping_moodys)
    # Create a unique Rating that averages the 3 Ratings and ignores missing values
    df['Rating'] = df[['Rating_Fitch_encoded', 'Rating_SP_encoded', 'Rating_Moodys_encoded']].mean(axis=1)

    # Add newly created boolean columns and 'Rating' to agg_dict with average
    agg_dict = {col: 'mean' for col in ['is_euro', 'is_fixed', 'Rating']}
    agg_dict.update({col: 'first' for col in ['Country', 'Classification']})
    agg_dict.update({num_col: ['min', 'max', 'median'] for num_col in numerical_columns})

    # Grouping by 'ISIN' and aggregating columns
    grouped_df = df.groupby('ISIN').agg(agg_dict).reset_index()

    # Flatten the multi-level column index
    grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]

    # Drop identical columns
    grouped_df = grouped_df.T.drop_duplicates().T

    # Set back data types to numerical when needed
    grouped_df = grouped_df.astype({col: 'float' for col in grouped_df.columns if col not in ['Classification_first', 'Country_first', 'ISIN_']})

    # Replace missing values with empty string
    grouped_df['Country_first'].replace({None: ''}, inplace=True)

    return grouped_df

In [ ]:
df_exp = preprocess_explainability(df_filled, cols_to_exclude)
df_exp['cluster'] = clusters
df_exp.set_index(['ISIN_'], inplace=True)
df_exp.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Function to train CatBoostClassifier for each cluster label
def train_catboost_classifier(df, cluster_labels, min_representation=100):
    classifiers = {}

    for label in cluster_labels:
        # Check if the cluster label is represented at least min_representation times
        if df['cluster'].value_counts().get(label, 0) < min_representation:
            print(f"Skipping cluster {label} as it has less than {min_representation} instances.")
            continue

        # Create binary labels for the current cluster
        lb = LabelBinarizer()
        binary_labels = lb.fit_transform(df['cluster'] == label).ravel()

        # Split the data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(
            df.drop('cluster', axis=1), binary_labels, test_size=0.2, random_state=42
        )

        # Initialize CatBoostClassifier
        clf = CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss')

        # Train the classifier
        clf.fit(X_train, y_train, cat_features=['Country_first', 'Classification_first'], verbose=False)

        # Make predictions on the test set
        y_pred = clf.predict(X_test)

        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        # Store the classifier and evaluation results
        classifiers[label] = {'classifier': clf, 'accuracy': accuracy, 'classification_report': report}

        # Retrain the model on the full data
        clf.fit(df.drop('cluster', axis=1), binary_labels, cat_features=['Country_first', 'Classification_first'], verbose=False)

        # Save the retrained model in classifiers dict
        classifiers[label]['classifier_full_data'] = clf

    return classifiers


In [ ]:
# Train CatBoostClassifier for each cluster label
cluster_classifiers = train_catboost_classifier(df_exp, cluster_labels=df_exp['cluster'].unique())

In [ ]:
def plot_shap_explainability(model, df):
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(df)

    # Display beeswarm SHAP plot
    #shap.plots.beeswarm(shap_values)
    shap.summary_plot(shap_values, df, plot_type="beeswarm")

In [ ]:
def plot_shap_explainability_corrected(model, df):
    
    df_to_plot = df.reset_index().copy()
    ISIN_list = df_to_plot.ISIN_.to_list()
    index = pd.Index(ISIN_list)
    df_to_plot.drop(columns=["ISIN_"], inplace=True)
    df_to_plot = df_to_plot.set_index(index)
    df_to_plot = df_to_plot.drop('cluster', axis=1)

    explainer = shap.TreeExplainer(model)
    shap_values = explainer(df_to_plot)
    shap.plots.beeswarm(shap_values)

In [ ]:
# Assuming cluster_classifiers is already defined
for label, info in cluster_classifiers.items():
    print(f"Classifier for Cluster {label}:")
    print(f"Accuracy: {info['accuracy']:.2f}")
    print("Classification Report:")
    print(info['classification_report'])
    print("\n")

    # Check if 'classifier_full_data' key exists in the dictionary
    if 'classifier_full_data' in info:
        print(f"Plotting SHAP explainability for Cluster {label}")
        # Access the retrained model on full data
        full_data_model = info['classifier_full_data']
        
        # Plot SHAP explainability
        plot_shap_explainability_corrected(full_data_model, df_exp)
        
        plt.show()  # Display the plot
        print("\n")
    else:
        print(f"No 'classifier_full_data' available for Cluster {label}\n")


### Recommending similar bonds

In [ ]:
df_clustering_filled

In [ ]:
df_to_test = df_clustering_filled.reset_index().copy()
ISIN_list = df_to_test.ISIN_.to_list()
index = pd.Index(ISIN_list)
df_to_test.drop(columns=["ISIN_"], inplace=True)
df_to_test = df_to_test.set_index(index)
df_to_test

In [14]:
from sklearn.metrics import pairwise_distances

def recommend_n_bonds(row_id, df, kmeans_model):
    # Get the cluster of the given row
    cluster_id = kmeans_model.predict([df.loc[row_id]])[0]
    # Get the indices of data points in the same cluster
    cluster_indices = np.where(kmeans_model.labels_ == cluster_id)[0]
    # Get the distances between the given row and all other points in the cluster
    distances = pairwise_distances(df.loc[[row_id]], df.iloc[cluster_indices], metric='euclidean')[0]
    # Sort indices based on distances and get the top 5 nearest indices
    sorted_indices = np.argsort(distances)
    top5_nearest_indices = cluster_indices[sorted_indices][:5]

    return top5_nearest_indices.tolist()

# Example usage:
#row_id_to_check = 0  # Replace with the desired row index
#top5_nearest_ids = recommend_n_bonds('AT0000383864', df_to_test, model)

#print(f"Top 5 nearest ids for row {row_id_to_check}: {top5_nearest_ids}")

In [ ]:
model.predict(df_clustering_filled.loc['XS2717309855'])

In [ ]:
top5_nearest_ids = recommend_n_bonds('XS2717309855', df_to_test, model)

print(f"Top 5 nearest ids for row {row_id_to_check}: {top5_nearest_ids}")

In [ ]:
df[(df['ISIN'] == 'XS2236363573')]

In [ ]:
df_test = df.copy()
df_test['B_Price'] = float(df['B_Price'])

In [ ]:
df_preprocessed[df_preprocessed['ISIN']=='XS2236363573']

## Unsupervised clustering

In [ ]:
import hdbscan

In [ ]:
pd.DataFrame(df_normalized, columns=df_clustering_filled.columns).head()

In [ ]:
plt.scatter(x=embedding[:,0], y=embedding[:,1])

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, min_samples=5, gen_min_span_tree=True)
clusters = clusterer.fit_predict(df_normalized)

In [ ]:
sns.scatterplot(x=embedding[:,0], y=embedding[:,1], hue=clusters, palette='dark')

In [ ]:
pd.Series(clusters).value_counts()

## Predicting for Natixis test

In [15]:
df_preprocessed.head()

,Deal_Date,ISIN,cusip,Instrument,Sales_Name,Sales_Initial,company_short_name,B_Price,B_Side,Total_Requested_Volume,Total_Traded_Volume_Natixis,Total_Traded_Volume_Away,Total_Traded_Volume,BloomIndustrySector,BloomIndustryGroup,BloomIndustrySubGroup,cdcissuerShortName,cdcissuer,Country,lb_Platform_2,maturity,Rating_Fitch,Rating_Moodys,Rating_SP,Ccy,Classification,Tier,AssumedMaturity,Coupon,Frequency,Type,MidPrice,MidYTM,YTWDate,SpreadvsBenchmarkMid,MidASWSpread,MidZSpread,GSpreadMid,MidModifiedDuration,MidConvexity,MidEffectiveDuration,MidEffectiveConvexity,Year_dealdate,Month_dealdate,Day_dealdate,Year_maturity,Month_maturity,Day_maturity,Days_to_Maturity
6,2022-11-10,NL0011220108,EK8199722,NETHER 0.25% 15 JUL 2025,heddy kerneves,HED,banque populaire de l'ouest,95,1,13061,0.0,13061.0,13061.0,Government,Sovereign,Sovereign,ROPAYSBAS,NETHERLANDS GOVERNMENT BOND,NETHERLANDS,EMEA,2025-07-15,AAA,Aaa,AAA,EUR,Government,UNKNOWN,2025-07-15,0.25,12,Fixed,95.125999,2.15064,NaT,64.479906,-72.215584,-74.569401,66.357435,2.60208,10.181840,2.47743,8.883440,2022,11,10,2025.0,7.0,15.0,978
7,2022-11-10,NL0011220108,EK8199722,NETHER 0.25% 15 JUL 2025,ian prechais,IAN,license de transport,95,0,136,0.0,136.0,136.0,Government,Sovereign,Sovereign,ROPAYSBAS,NETHERLANDS GOVERNMENT BOND,NETHERLANDS,EMEA,2025-07-15,AAA,Aaa,AAA,EUR,Government,UNKNOWN,2025-07-15,0.25,12,Fixed,95.125999,2.15064,NaT,64.479906,-72.215584,-74.569401,66.357435,2.60208,10.181840,2.47743,8.883440,2022,11,10,2025.0,7.0,15.0,978
8,2022-11-10,NL0011220108,EK8199722,NETHER 0.25% 15 JUL 2025,ian prechais,IAN,bp val de france,95,0,1224490,0.0,1224490.0,1224490.0,Government,Sovereign,Sovereign,ROPAYSBAS,NETHERLANDS GOVERNMENT BOND,NETHERLANDS,EMEA,2025-07-15,AAA,Aaa,AAA,EUR,Government,UNKNOWN,2025-07-15,0.25,12,Fixed,95.125999,2.15064,NaT,64.479906,-72.215584,-74.569401,66.357435,2.60208,10.181840,2.47743,8.883440,2022,11,10,2025.0,7.0,15.0,978
9,2022-11-10,NL0011220108,EK8199722,NETHER 0.25% 15 JUL 2025,elisene arslane,ELI,hsbc,95,0,91837,0.0,91837.0,91837.0,Government,Sovereign,Sovereign,ROPAYSBAS,NETHERLANDS GOVERNMENT BOND,NETHERLANDS,EMEA,2025-07-15,AAA,Aaa,AAA,EUR,Government,UNKNOWN,2025-07-15,0.25,12,Fixed,95.125999,2.15064,NaT,64.479906,-72.215584,-74.569401,66.357435,2.60208,10.181840,2.47743,8.883440,2022,11,10,2025.0,7.0,15.0,978
12,2023-10-18,IT0005530032,ZM3378175,BTPS 4.45% 01 SEP 2043,ian prechais,IAN,societe generale mer,92,0,2041,0.0,2041.0,2041.0,Government,Sovereign,Sovereign,REPITALIE,BTP,ITALY,EMEA,2043-09-01,BBB,Baa3,BBB,EUR,Government,UNKNOWN,2043-09-01,4.45,6,Fixed,90.022003,5.26552,NaT,213.429138,173.721893,183.643234,217.344910,12.58312,212.362656,11.40296,192.445251,2023,10,18,2043.0,9.0,1.0,7258


In [20]:
df_bond = df[df['ISIN']=='XS2236363573']
df_bond.head()

,Deal_Date,ISIN,cusip,Instrument,Sales_Name,Sales_Initial,company_short_name,B_Price,B_Side,Total_Requested_Volume,Total_Traded_Volume_Natixis,Total_Traded_Volume_Away,Total_Traded_Volume,BloomIndustrySector,BloomIndustryGroup,BloomIndustrySubGroup,cdcissuerShortName,cdcissuer,Country,lb_Platform_2,maturity,Rating_Fitch,Rating_Moodys,Rating_SP,Cusip,Ccy,Classification,Tier,Maturity,AssumedMaturity,Coupon,Frequency,Type,MidPrice,MidYTM,YTWDate,SpreadvsBenchmarkMid,MidASWSpread,MidZSpread,GSpreadMid,MidModifiedDuration,MidConvexity,MidEffectiveDuration,MidEffectiveConvexity
173729,2023-12-14,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,Constantin Durie,CON,Scp Laureau-Jeannerot,0.0,NATIXIS SELL,40816,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24 00:00:00.000,NaN,WR,E04648AU0,EUR,Technology,NaN,09/24/2028 00:00:00,NaN,1.875,12M,Fixed,94.926643,3.03277,09/24/2028 00:00:00,106.666039,47.90052,48.5846,NaN,4.44633,24.584961,4.57407,5.77398
239414,2022-09-26,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,Angelle Vandeneynde,ANG,Garantie Oseo,0.000000,NATIXIS SELL,163265,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24 00:00:00.000,NaN,WR,E04648AU0,EUR,Technology,NaN,09/24/2028 00:00:00,NaN,1.875,12M,Fixed,87.281853,4.33122,09/24/2028 00:00:00,228.421448,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239415,2022-09-26,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,Melvyn Chorlet,MEL,Cegc,0.0,NATIXIS SELL,163265,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24 00:00:00.000,NaN,WR,E04648AU0,EUR,Technology,NaN,09/24/2028 00:00:00,NaN,1.875,12M,Fixed,87.281853,4.33122,09/24/2028 00:00:00,228.421448,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288069,2023-04-17,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,Constantin Durie,CON,Scp Laureau-Jeannerot,0.0,NATIXIS SELL,204082,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24 00:00:00.000,NaN,WR,E04648AU0,EUR,Technology,NaN,09/24/2028 00:00:00,NaN,1.875,12M,Fixed,90.753853,3.78817,09/24/2028 00:00:00,136.514847,NaN,NaN,NaN,4.95791,NaN,NaN,NaN
314443,2022-12-23,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,Florindo Garmy,FLO,License De Transport,0.0,NATIXIS SELL,40816,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24 00:00:00.000,NaN,WR,E04648AU0,EUR,Technology,NaN,09/24/2028 00:00:00,NaN,1.875,12M,Fixed,90.151421,3.81675,09/24/2028 00:00:00,146.680328,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def preprocess_bond(df):
    """
    Preprocesses the input DataFrame with the following steps:
    1. Converts 'Deal_Date', 'maturity', 'AssumedMaturity', 'YTWDate' columns to datetime.
    2. Converts 'B_Side' column to boolean (1 for 'NATIXIS BUY', 0 for 'NATIXIS SELL').
    3. Converts 'B_Price' and 'Total_Requested_Volume' columns to integers.
    4. Fills null values in 'Tier', 'AssumedMaturity', and 'YTWDate' columns with 'UNKNOWN'.
    5. Converts 'Frequency' feature values into integers (removing 'M' from the end).
    6. Drops the unsused 'Cusip' column.

    Parameters:
    - df (DataFrame): Input DataFrame.

    Returns:
    - DataFrame: Processed DataFrame.
    """

    df = df.copy()

    # Shift back the columns to the correct place
    column_names = df.columns
    # Find the index of 'cdIssuerShortName' and 'maturity'
    cd_issuer_index = column_names.get_loc('cdcissuerShortName')
    maturity_index = column_names.get_loc('maturity')
    # Loop through each column and shift the data to the left
    for i in range(cd_issuer_index, maturity_index + 1):
        df.iloc[:, i] = df.iloc[:, i + 1]

    # Replace empty column with nans
    df.iloc[:, maturity_index+1] = np.nan

    # Convert 'B_Price', 'Total_Requested_Volume', 'Frequency' to integers
    df['Frequency'] = df['Frequency'].str.replace('M', '')
    numerical_columns = ['B_Price', 'Total_Requested_Volume', 'Frequency']
    #df.dropna(subset=numerical_columns, inplace=True)
    for column in numerical_columns:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype(int)

    # Fix the error in the B_Price column
    #df = df[df['B_Price'] >= 20]

    # Replace NaT with null values in the 'Maturity' column
    df['maturity'].replace({pd.NaT: np.nan}, inplace=True)

    # Convert 'Deal_Date', 'maturity', 'AssumedMaturity', 'YTWDate' to datetime
    df['Deal_Date'] = pd.to_datetime(df['Deal_Date'])
    df['maturity'] = pd.to_datetime(df['maturity'], errors='coerce',  format='%Y-%m-%d %H:%M:%S.%f')
    df['AssumedMaturity'] = pd.to_datetime(df['AssumedMaturity'], errors='coerce')
    df['YTWDate'] = pd.to_datetime(df['YTWDate'], errors='coerce')

    # Add year, month, day for clustering 
    df['Year_dealdate'] = df['Deal_Date'].dt.year
    df['Month_dealdate'] = df['Deal_Date'].dt.month
    df['Day_dealdate'] = df['Deal_Date'].dt.day
    df['Year_maturity'] = df['maturity'].dt.year
    df['Month_maturity'] = df['maturity'].dt.month
    df['Day_maturity'] = df['maturity'].dt.day

    # Delete maturities smaller than 2021 (as deal dates starts in 2021)
    #df = df[df['maturity'].dt.year >= 2021]

    # Compute number of days between maturity and deal date
    df['Days_to_Maturity'] = (df['maturity'] - df['Deal_Date']).dt.days

    # Replace null values in 'AssumedMaturity' with values from 'Maturity'
    df['AssumedMaturity'] = df['AssumedMaturity'].fillna(df['Maturity'])

    # Convert 'B_Side' column to boolean (1 for 'NATIXIS BUY', 0 for 'NATIXIS SELL')
    df = df[df['B_Side'].isin(['NATIXIS SELL', 'NATIXIS BUY'])]
    df['B_Side'] = df['B_Side'].replace({'NATIXIS BUY': 1, 'NATIXIS SELL': 0})

    # Convert null values of 'Tier'
    df['Tier'].fillna('UNKNOWN', inplace=True)

    # Lower string names 
    df['Sales_Name'] = df['Sales_Name'].str.lower()
    df['company_short_name'] = df['company_short_name'].str.lower()

    # Drop unused columns
    columns_to_drop = ['Cusip', 'Maturity']
    df.drop(columns=columns_to_drop, inplace=True)

    return df

In [22]:
df_bond = preprocess_bond(df_bond)

In [23]:
df_bond.head()

,Deal_Date,ISIN,cusip,Instrument,Sales_Name,Sales_Initial,company_short_name,B_Price,B_Side,Total_Requested_Volume,Total_Traded_Volume_Natixis,Total_Traded_Volume_Away,Total_Traded_Volume,BloomIndustrySector,BloomIndustryGroup,BloomIndustrySubGroup,cdcissuerShortName,cdcissuer,Country,lb_Platform_2,maturity,Rating_Fitch,Rating_Moodys,Rating_SP,Ccy,Classification,Tier,AssumedMaturity,Coupon,Frequency,Type,MidPrice,MidYTM,YTWDate,SpreadvsBenchmarkMid,MidASWSpread,MidZSpread,GSpreadMid,MidModifiedDuration,MidConvexity,MidEffectiveDuration,MidEffectiveConvexity,Year_dealdate,Month_dealdate,Day_dealdate,Year_maturity,Month_maturity,Day_maturity,Days_to_Maturity
173729,2023-12-14,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,constantin durie,CON,scp laureau-jeannerot,0,0,40816,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,94.926643,3.03277,2028-09-24,106.666039,47.90052,48.5846,NaN,4.44633,24.584961,4.57407,5.77398,2023,12,14,2028,6,24,1654
239414,2022-09-26,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,angelle vandeneynde,ANG,garantie oseo,0,0,163265,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,87.281853,4.33122,2028-09-24,228.421448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,9,26,2028,6,24,2098
239415,2022-09-26,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,melvyn chorlet,MEL,cegc,0,0,163265,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,87.281853,4.33122,2028-09-24,228.421448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,9,26,2028,6,24,2098
288069,2023-04-17,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,constantin durie,CON,scp laureau-jeannerot,0,0,204082,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,90.753853,3.78817,2028-09-24,136.514847,NaN,NaN,NaN,4.95791,NaN,NaN,NaN,2023,4,17,2028,6,24,1895
314443,2022-12-23,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,florindo garmy,FLO,license de transport,0,0,40816,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,90.151421,3.81675,2028-09-24,146.680328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,12,23,2028,6,24,2010


In [24]:
df_bond = pd.concat([df_preprocessed, df_bond], axis=0)
df_bond.tail()

,Deal_Date,ISIN,cusip,Instrument,Sales_Name,Sales_Initial,company_short_name,B_Price,B_Side,Total_Requested_Volume,Total_Traded_Volume_Natixis,Total_Traded_Volume_Away,Total_Traded_Volume,BloomIndustrySector,BloomIndustryGroup,BloomIndustrySubGroup,cdcissuerShortName,cdcissuer,Country,lb_Platform_2,maturity,Rating_Fitch,Rating_Moodys,Rating_SP,Ccy,Classification,Tier,AssumedMaturity,Coupon,Frequency,Type,MidPrice,MidYTM,YTWDate,SpreadvsBenchmarkMid,MidASWSpread,MidZSpread,GSpreadMid,MidModifiedDuration,MidConvexity,MidEffectiveDuration,MidEffectiveConvexity,Year_dealdate,Month_dealdate,Day_dealdate,Year_maturity,Month_maturity,Day_maturity,Days_to_Maturity
602227,2022-08-18,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,susanne tricart,SUS,hsbc,0,0,244898,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,93.084160,3.13980,2028-09-24,226.418289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,8,18,2028.0,6.0,24.0,2137
613416,2022-12-16,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,roberthe chakri,ROB,credit cooperatif,0,1,163265,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,90.851280,3.66398,2028-09-24,148.157104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,12,16,2028.0,6.0,24.0,2017
614587,2022-09-14,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,angelle vandeneynde,ANG,garantie oseo,0,0,367347,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,89.552307,3.85184,2028-09-24,229.040924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,9,14,2028.0,6.0,24.0,2110
622137,2023-01-19,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,susanne tricart,SUS,hsbc,0,0,285714,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,92.626801,3.32215,2028-09-24,128.359955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,1,19,2028.0,6.0,24.0,1983
626904,2022-09-02,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,angelle vandeneynde,ANG,garantie oseo,0,0,40816,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,90.283951,3.69649,2028-09-24,239.573669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,9,2,2028.0,6.0,24.0,2122


In [25]:
df_bond_filled = complete_nan_values(df_bond)
df_bond.tail()

,Deal_Date,ISIN,cusip,Instrument,Sales_Name,Sales_Initial,company_short_name,B_Price,B_Side,Total_Requested_Volume,Total_Traded_Volume_Natixis,Total_Traded_Volume_Away,Total_Traded_Volume,BloomIndustrySector,BloomIndustryGroup,BloomIndustrySubGroup,cdcissuerShortName,cdcissuer,Country,lb_Platform_2,maturity,Rating_Fitch,Rating_Moodys,Rating_SP,Ccy,Classification,Tier,AssumedMaturity,Coupon,Frequency,Type,MidPrice,MidYTM,YTWDate,SpreadvsBenchmarkMid,MidASWSpread,MidZSpread,GSpreadMid,MidModifiedDuration,MidConvexity,MidEffectiveDuration,MidEffectiveConvexity,Year_dealdate,Month_dealdate,Day_dealdate,Year_maturity,Month_maturity,Day_maturity,Days_to_Maturity
602227,2022-08-18,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,susanne tricart,SUS,hsbc,0,0,244898,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,93.084160,3.13980,2028-09-24,226.418289,110.481620,87.764959,179.161051,2.385497,17.839323,2.228408,14.499675,2022,8,18,2028.0,6.0,24.0,2137
613416,2022-12-16,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,roberthe chakri,ROB,credit cooperatif,0,1,163265,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,90.851280,3.66398,2028-09-24,148.157104,110.481620,87.764959,179.161051,2.385497,17.839323,2.228408,14.499675,2022,12,16,2028.0,6.0,24.0,2017
614587,2022-09-14,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,angelle vandeneynde,ANG,garantie oseo,0,0,367347,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,89.552307,3.85184,2028-09-24,229.040924,110.481620,87.764959,179.161051,2.385497,17.839323,2.228408,14.499675,2022,9,14,2028.0,6.0,24.0,2110
622137,2023-01-19,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,susanne tricart,SUS,hsbc,0,0,285714,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,92.626801,3.32215,2028-09-24,128.359955,79.614797,86.028474,158.674089,4.360570,28.182018,3.732556,20.946393,2023,1,19,2028.0,6.0,24.0,1983
626904,2022-09-02,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,angelle vandeneynde,ANG,garantie oseo,0,0,40816,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,90.283951,3.69649,2028-09-24,239.573669,110.481620,87.764959,179.161051,2.385497,17.839323,2.228408,14.499675,2022,9,2,2028.0,6.0,24.0,2122


We impute the missing values for price.

In [26]:
# Assuming 'B_Price' and 'MidPrice' are columns in your DataFrame 'df'
correlation = df_bond['B_Price'].corr(df_bond['MidPrice'])

print(f"Correlation between B_Price and MidPrice: {correlation:.4f}")

Correlation between B_Price and MidPrice: 0.9939


In [27]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Assuming 'B_Price' is the independent variable and 'MidPrice' is the dependent variable
X = df_bond[['MidPrice']]
y = df_bond['B_Price']

# Create a linear regression model
model = LinearRegression()

# Fit the model
model.fit(X, y)

# Print the coefficients
p_intercept = model.intercept_
p_slope = model.coef_[0]

print(f"Intercept: {p_intercept:.4f}")
print(f"Slope (Coefficient for B_Price): {p_slope:.4f}")


Intercept: 0.5937
Slope (Coefficient for B_Price): 0.9896


We now set the missing prices as price = 0.9896 * MidPrice + 0.5937

In [29]:
# Use a lambda function to calculate the predicted values
fill_zero = lambda mid_price: p_intercept + p_slope * mid_price

# Create a boolean mask for values equal to 0 in 'B_Price'
mask = df_bond['B_Price'] == 0

# Apply the lambda function to replace zero values in 'B_Price'
df_bond.loc[mask, 'B_Price'] = df_bond.loc[mask, 'MidPrice'].apply(fill_zero)

df_bond.tail()

C:\Users\gascm\AppData\Local\Temp\ipykernel_2544\1168339447.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[94.53060352 86.96551694 86.96551694 90.40131823 89.80516689 91.6134166
 89.21747894 91.20201767 86.52952815 91.20752906 90.86486398 90.81403083
 94.29781897 91.00652183 88.41838007 89.14179169 91.6311059  90.30056547
 89.88535386 89.7641863  87.15453501 92.20086296 87.73576029 87.14329329
 90.42897333 93.58406361 89.53691314 93.64983036 94.321752   90.76075908
 90.91646721 91.44000408 90.65996102 91.09628956 91.61574196 87.60213548
 89.6073306  91.35869218 91.42071421 90.73540668 92.70732961 90.49772982
 89.21229975 92.25473869 89.93631536]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_bond.loc[mask, 'B_Price'] = df_bond.loc[mask, 'MidPrice'].apply(fill_zero)


,Deal_Date,ISIN,cusip,Instrument,Sales_Name,Sales_Initial,company_short_name,B_Price,B_Side,Total_Requested_Volume,Total_Traded_Volume_Natixis,Total_Traded_Volume_Away,Total_Traded_Volume,BloomIndustrySector,BloomIndustryGroup,BloomIndustrySubGroup,cdcissuerShortName,cdcissuer,Country,lb_Platform_2,maturity,Rating_Fitch,Rating_Moodys,Rating_SP,Ccy,Classification,Tier,AssumedMaturity,Coupon,Frequency,Type,MidPrice,MidYTM,YTWDate,SpreadvsBenchmarkMid,MidASWSpread,MidZSpread,GSpreadMid,MidModifiedDuration,MidConvexity,MidEffectiveDuration,MidEffectiveConvexity,Year_dealdate,Month_dealdate,Day_dealdate,Year_maturity,Month_maturity,Day_maturity,Days_to_Maturity
602227,2022-08-18,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,susanne tricart,SUS,hsbc,92.707330,0,244898,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,93.084160,3.13980,2028-09-24,226.418289,110.481620,87.764959,179.161051,2.385497,17.839323,2.228408,14.499675,2022,8,18,2028.0,6.0,24.0,2137
613416,2022-12-16,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,roberthe chakri,ROB,credit cooperatif,90.497730,1,163265,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,90.851280,3.66398,2028-09-24,148.157104,110.481620,87.764959,179.161051,2.385497,17.839323,2.228408,14.499675,2022,12,16,2028.0,6.0,24.0,2017
614587,2022-09-14,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,angelle vandeneynde,ANG,garantie oseo,89.212300,0,367347,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,89.552307,3.85184,2028-09-24,229.040924,110.481620,87.764959,179.161051,2.385497,17.839323,2.228408,14.499675,2022,9,14,2028.0,6.0,24.0,2110
622137,2023-01-19,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,susanne tricart,SUS,hsbc,92.254739,0,285714,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,92.626801,3.32215,2028-09-24,128.359955,79.614797,86.028474,158.674089,4.360570,28.182018,3.732556,20.946393,2023,1,19,2028.0,6.0,24.0,1983
626904,2022-09-02,XS2236363573,E04648AU0,749E - AMSSM 1.875% 24 SEP 202,angelle vandeneynde,ANG,garantie oseo,89.936315,0,40816,0.0,0.0,0.0,Consumer,Non-cyclical,Commercial Services,AMADEUCMSA,AMADEUS IT GROUP SA,SPAIN,EMEA,2028-06-24,NaN,NaN,WR,EUR,Technology,UNKNOWN,2028-09-24,1.875,12,Fixed,90.283951,3.69649,2028-09-24,239.573669,110.481620,87.764959,179.161051,2.385497,17.839323,2.228408,14.499675,2022,9,2,2028.0,6.0,24.0,2122


In [30]:
cols_to_exclude = ['Deal_Date', 'cusip', 'B_Side', 'Instrument', 'Sales_Name', 'Sales_Initial', 'company_short_name',
                   'Total_Requested_Volume', 'Total_Traded_Volume_Natixis', 'Total_Traded_Volume_Away', 'Total_Traded_Volume',
                   'cdissuer', 'Tier', 'Year_dealdate', 'Month_dealdate','Day_dealdate', 'Days_to_Maturity',
                   'cdissuerShortName', 'lb_Platform_2']
df_bond_clustering = preprocess_clustering(df_bond, cols_to_exclude)

In [31]:
df_bond_clustering[df_bond_clustering['ISIN_']=='XS2236363573']

,ISIN_,is_euro_mean,is_fixed_mean,Rating_mean,is_FRANCE_first,is_ITALY_first,is_GERMANY_first,is_NETHERLANDS_first,is_SPAIN_first,is_Financials_first,is_Government_first,is_Industrials_first,is_Utilities_first,B_Price_min,B_Price_max,B_Price_median,Coupon_min,Coupon_max,Coupon_median,Frequency_min,Frequency_max,MidPrice_min,MidPrice_max,MidPrice_median,MidYTM_min,MidYTM_max,MidYTM_median,SpreadvsBenchmarkMid_min,SpreadvsBenchmarkMid_max,SpreadvsBenchmarkMid_median,MidASWSpread_min,MidASWSpread_max,MidASWSpread_median,MidZSpread_min,MidZSpread_max,MidZSpread_median,GSpreadMid_min,GSpreadMid_max,GSpreadMid_median,MidModifiedDuration_min,MidModifiedDuration_max,MidModifiedDuration_median,MidConvexity_min,MidConvexity_max,MidConvexity_median,MidEffectiveDuration_min,MidEffectiveDuration_max,MidEffectiveDuration_median,MidEffectiveConvexity_min,MidEffectiveConvexity_max,MidEffectiveConvexity_median,Year_maturity_min,Month_maturity_min,Day_maturity_min
3985,XS2236363573,0.977778,1.0,NaN,False,False,False,False,True,False,False,False,False,86.529528,94.530604,90.735407,1.875,1.875,1.875,12,12,86.84127,94.926643,91.091461,2.82396,4.4393,3.70355,106.666039,240.187836,148.157104,40.52055,110.48162,110.48162,41.70961,87.764959,87.764959,158.674089,179.161051,179.161051,2.385497,5.00435,2.385497,17.839323,28.182018,17.839323,2.228408,4.57407,2.228408,5.77398,20.946393,14.499675,2028.0,6.0,24.0


In [32]:
missing_values = df_bond_clustering.isnull().sum()
missing_values[missing_values!=0] 

Rating_mean    292
dtype: int64

In [33]:
df_bond_clustering_filled = df_bond_clustering.copy()
df_bond_clustering_filled['Rating_mean'] = df_bond_clustering['Rating_mean'].fillna(df_bond_clustering['Rating_mean'].median())

In [36]:
df_normalized = df_bond_clustering_filled.drop(columns=['ISIN_'])

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_normalized = scaler.fit_transform(df_normalized)

In [39]:
pd.DataFrame(df_normalized, columns=df_bond_clustering_filled.columns[1:]).head()

,is_euro_mean,is_fixed_mean,Rating_mean,is_FRANCE_first,is_ITALY_first,is_GERMANY_first,is_NETHERLANDS_first,is_SPAIN_first,is_Financials_first,is_Government_first,is_Industrials_first,is_Utilities_first,B_Price_min,B_Price_max,B_Price_median,Coupon_min,Coupon_max,Coupon_median,Frequency_min,Frequency_max,MidPrice_min,MidPrice_max,MidPrice_median,MidYTM_min,MidYTM_max,MidYTM_median,SpreadvsBenchmarkMid_min,SpreadvsBenchmarkMid_max,SpreadvsBenchmarkMid_median,MidASWSpread_min,MidASWSpread_max,MidASWSpread_median,MidZSpread_min,MidZSpread_max,MidZSpread_median,GSpreadMid_min,GSpreadMid_max,GSpreadMid_median,MidModifiedDuration_min,MidModifiedDuration_max,MidModifiedDuration_median,MidConvexity_min,MidConvexity_max,MidConvexity_median,MidEffectiveDuration_min,MidEffectiveDuration_max,MidEffectiveDuration_median,MidEffectiveConvexity_min,MidEffectiveConvexity_max,MidEffectiveConvexity_median,Year_maturity_min,Month_maturity_min,Day_maturity_min
0,0.331502,0.312509,1.055387,-0.537944,-0.256656,-0.455426,-0.244747,-0.241464,-1.111714,1.528783,-0.195453,-0.212225,1.920841,3.330468,2.106067,2.698808,2.697055,2.698422,0.376974,0.376271,1.913631,3.438522,2.088718,-1.571056,-0.499470,-0.497146,-0.742703,-0.666415,-0.708261,-1.218895,-0.954227,-1.107721,-1.260771,-0.426653,-0.936953,-0.859735,-0.666796,-0.701550,-0.290544,-0.140668,-0.280095,-0.258951,-0.211629,-0.252074,-0.146134,0.267667,0.001516,-0.042689,-0.055699,-0.053407,-0.188324,0.313215,-0.178296
1,0.331502,0.312509,1.055387,-0.537944,-0.256656,-0.455426,-0.244747,-0.241464,-1.111714,1.528783,-0.195453,-0.212225,0.810705,3.862772,1.998853,1.425845,1.424435,1.425382,0.376974,0.376271,1.214198,4.022359,1.935115,-0.911050,-0.074066,-0.326084,-0.385613,-0.428961,-0.408676,-0.466999,-0.319853,-0.390773,-0.497525,-0.160247,-0.324796,-0.518561,-0.355468,-0.410982,1.353961,1.483555,1.377771,0.721946,0.821827,0.751155,2.183512,2.709872,2.378726,-0.042666,-0.055670,-0.053385,1.282797,-0.857534,-0.178296
2,0.331502,0.312509,1.055387,-0.537944,-0.256656,-0.455426,-0.244747,-0.241464,-1.111714,1.528783,-0.195453,-0.212225,1.214391,1.946476,1.248355,1.850166,1.848642,1.849728,0.376974,0.376271,1.212746,1.949409,1.286747,-1.315728,-0.443046,-0.492046,-0.928232,-0.555838,-0.522281,-1.179339,-1.043319,-1.360615,-1.283571,-0.472505,-1.179086,-1.007732,-0.767612,-0.865302,-0.563119,-0.405411,-0.517334,-0.322977,-0.281475,-0.310036,-0.523957,-0.199481,-0.423757,-0.042690,-0.055701,-0.053409,-0.335436,-0.857534,-0.178296
3,0.331502,0.312509,1.055387,-0.537944,-0.256656,-0.455426,-0.244747,-0.241464,-1.111714,1.528783,-0.195453,-0.212225,1.012548,6.204912,2.642137,1.213684,1.212331,1.213208,0.376974,0.376271,0.818656,6.391203,2.497268,-0.768259,-0.065694,-0.393000,-0.352273,-0.341492,-0.423110,0.325603,0.527146,0.448336,0.240061,0.141089,0.324106,-0.312365,-0.226786,-0.322561,3.636318,4.381920,3.934890,4.557273,5.436327,4.914243,5.330228,8.285194,6.408000,-0.042574,-0.055515,-0.053283,4.960598,-1.442908,1.112499
4,0.331502,0.312509,1.055387,-0.537944,-0.256656,-0.455426,-0.244747,-0.241464,-1.111714,1.528783,-0.195453,-0.212225,0.911626,0.562485,0.819499,0.971215,0.969927,0.970725,0.376974,0.376271,0.855629,0.631253,0.792629,-2.183830,-2.181398,-2.217633,-1.800617,-0.986766,-1.529866,-0.332107,-0.751341,-0.682084,-2.183490,-0.656536,-2.114639,-1.706394,-0.886866,-1.195310,-1.050965,-1.092183,-1.088731,-0.377461,-0.393814,-0.385816,-1.210669,-1.071138,-1.206528,-0.042691,-0.055704,-0.053411,-0.923884,1.483964,0.643119


In [40]:
from sklearn.cluster import KMeans

In [41]:
clusterer = KMeans(n_clusters=7)
clusterer.fit(df_normalized)
clusters = clusterer.predict(df_normalized)

c:\Users\gascm\OneDrive\Documents\HEC\DSB_2\Natixis\.natixis\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [42]:
df_bond_clustering_filled['cluster'] = clusters

In [43]:
df_bond_clustering_filled.head()

,ISIN_,is_euro_mean,is_fixed_mean,Rating_mean,is_FRANCE_first,is_ITALY_first,is_GERMANY_first,is_NETHERLANDS_first,is_SPAIN_first,is_Financials_first,is_Government_first,is_Industrials_first,is_Utilities_first,B_Price_min,B_Price_max,B_Price_median,Coupon_min,Coupon_max,Coupon_median,Frequency_min,Frequency_max,MidPrice_min,MidPrice_max,MidPrice_median,MidYTM_min,MidYTM_max,MidYTM_median,SpreadvsBenchmarkMid_min,SpreadvsBenchmarkMid_max,SpreadvsBenchmarkMid_median,MidASWSpread_min,MidASWSpread_max,MidASWSpread_median,MidZSpread_min,MidZSpread_max,MidZSpread_median,GSpreadMid_min,GSpreadMid_max,GSpreadMid_median,MidModifiedDuration_min,MidModifiedDuration_max,MidModifiedDuration_median,MidConvexity_min,MidConvexity_max,MidConvexity_median,MidEffectiveDuration_min,MidEffectiveDuration_max,MidEffectiveDuration_median,MidEffectiveConvexity_min,MidEffectiveConvexity_max,MidEffectiveConvexity_median,Year_maturity_min,Month_maturity_min,Day_maturity_min,cluster
0,AT0000383864,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,110.0,126.0,112.0,6.25,6.25,6.25,12,12,110.419762,127.079163,112.316982,0.6494,3.23769,2.94855,14.74143,48.205238,31.070089,-92.459953,-23.11165,-36.86031,-80.192131,-21.09169,-34.07125,7.43871,51.626171,35.860584,3.16226,4.41606,3.498495,13.70098,25.4069,16.758875,3.25192,5.63155,4.03505,11.2101,32.001419,19.12505,2027.0,7.0,15.0,1
1,AT0000A04967,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,99.0,131.0,111.0,4.15,4.15,4.15,12,12,103.552841,132.494583,110.889221,1.63245,3.80473,3.166545,50.77877,75.725662,62.114321,-19.249849,30.029989,14.9261,-17.64529,27.424015,13.593732,40.751419,85.924889,64.691807,9.97172,11.42595,10.46476,126.879807,166.019135,140.971497,10.33749,15.32064,11.76933,125.579231,216.827042,155.175697,2037.0,3.0,15.0,1
2,AT0000A0DXC2,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,103.0,113.0,104.0,4.85,4.85,4.85,12,12,103.538582,113.266792,104.862549,1.0297,3.3129,2.95505,-3.98206,61.020901,50.342121,-88.60849,-30.5749,-55.127319,-82.060589,-29.441706,-52.924715,-7.01188,40.519329,19.612419,2.0336,3.27347,2.50163,6.31344,15.90359,9.58242,2.10278,3.77821,2.65142,4.62418,16.69602,9.41744,2026.0,3.0,15.0,1
3,AT0000A0U299,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,101.0,153.0,117.0,3.8,3.8,3.8,12,12,99.669472,154.466949,116.114517,1.84513,3.81589,3.08127,54.143429,85.863159,60.618561,57.923691,100.983322,75.536491,42.798744,82.300621,64.11986,60.884682,100.101662,73.465332,19.42235,23.934839,21.209641,569.411377,793.869263,656.415588,19.908159,37.439911,24.87862,583.653931,1222.869385,760.366577,2062.0,1.0,26.0,4
4,AT0000A0U3T4,1.0,1.0,21.0,False,False,False,False,False,False,True,False,False,100.0,100.0,100.0,3.4,3.4,3.4,12,12,100.032471,101.040138,100.26965,-0.2633,0.99577,0.75603,-92.022652,11.07714,-54.067989,-6.115799,-6.115799,-6.115799,-155.807577,-62.955996,-125.770611,-75.230278,27.381149,-13.132235,0.01356,0.30947,0.10065,0.02686,0.6187,0.199865,0.01416,0.32004,0.104665,0.00039,0.19814,0.021345,2022.0,11.0,22.0,1


In [44]:
df_bond_clustering_filled[df_bond_clustering_filled['ISIN_']=='XS2236363573']

,ISIN_,is_euro_mean,is_fixed_mean,Rating_mean,is_FRANCE_first,is_ITALY_first,is_GERMANY_first,is_NETHERLANDS_first,is_SPAIN_first,is_Financials_first,is_Government_first,is_Industrials_first,is_Utilities_first,B_Price_min,B_Price_max,B_Price_median,Coupon_min,Coupon_max,Coupon_median,Frequency_min,Frequency_max,MidPrice_min,MidPrice_max,MidPrice_median,MidYTM_min,MidYTM_max,MidYTM_median,SpreadvsBenchmarkMid_min,SpreadvsBenchmarkMid_max,SpreadvsBenchmarkMid_median,MidASWSpread_min,MidASWSpread_max,MidASWSpread_median,MidZSpread_min,MidZSpread_max,MidZSpread_median,GSpreadMid_min,GSpreadMid_max,GSpreadMid_median,MidModifiedDuration_min,MidModifiedDuration_max,MidModifiedDuration_median,MidConvexity_min,MidConvexity_max,MidConvexity_median,MidEffectiveDuration_min,MidEffectiveDuration_max,MidEffectiveDuration_median,MidEffectiveConvexity_min,MidEffectiveConvexity_max,MidEffectiveConvexity_median,Year_maturity_min,Month_maturity_min,Day_maturity_min,cluster
3985,XS2236363573,0.977778,1.0,17.5,False,False,False,False,True,False,False,False,False,86.529528,94.530604,90.735407,1.875,1.875,1.875,12,12,86.84127,94.926643,91.091461,2.82396,4.4393,3.70355,106.666039,240.187836,148.157104,40.52055,110.48162,110.48162,41.70961,87.764959,87.764959,158.674089,179.161051,179.161051,2.385497,5.00435,2.385497,17.839323,28.182018,17.839323,2.228408,4.57407,2.228408,5.77398,20.946393,14.499675,2028.0,6.0,24.0,1


In [ ]:
from sklearn.metrics import pairwise_distances

def recommend_n_bonds(row_id, df, kmeans_model):
    # Get the cluster of the given row
    cluster_id = kmeans_model.predict([df.loc[row_id]])[0]
    # Get the indices of data points in the same cluster
    cluster_indices = np.where(kmeans_model.labels_ == cluster_id)[0]
    # Get the distances between the given row and all other points in the cluster
    distances = pairwise_distances(df.loc[[row_id]], df.iloc[cluster_indices], metric='euclidean')[0]
    # Sort indices based on distances and get the top 5 nearest indices
    sorted_indices = np.argsort(distances)
    top5_nearest_indices = cluster_indices[sorted_indices][:5]

    return top5_nearest_indices.tolist()

# Example usage:
#row_id_to_check = 0  # Replace with the desired row index
#top5_nearest_ids = recommend_n_bonds('AT0000383864', df_to_test, model)

#print(f"Top 5 nearest ids for row {row_id_to_check}: {top5_nearest_ids}")

In [45]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

def get_nearest_rows(df_normalized, isin_string):
    # Find the index of the given ISIN string in df_bond_clustering_filled
    index = df_bond_clustering_filled[df_bond_clustering_filled['ISIN_'] == isin_string].index[0]
    
    # Calculate Euclidean distances between the selected row and all other rows
    distances = euclidean_distances(df_normalized, [df_normalized[index]])
    
    # Get the indices of the 5 nearest rows (excluding the row itself)
    nearest_indices = np.argsort(distances.flatten())[1:6]
    
    # Retrieve the corresponding rows from the original DataFrame
    nearest_rows = df_bond_clustering_filled.iloc[nearest_indices]
    
    return nearest_rows

# Example usage:
isin_to_search = 'XS2236363573'
result = get_nearest_rows(df_normalized, isin_to_search)


             ISIN_ is_euro_mean is_fixed_mean  Rating_mean is_FRANCE_first  \
3939  XS2194370727          1.0           1.0    16.666667           False   
3590  XS1968846532          1.0           1.0    15.333333           False   
4457  XS2484587048          1.0           1.0    13.333333           False   
3653  XS2013574038          1.0           1.0    15.333333           False   
3654  XS2013745703          1.0           1.0    11.666667           False   

     is_ITALY_first is_GERMANY_first is_NETHERLANDS_first is_SPAIN_first  \
3939          False            False                False           True   
3590          False            False                False           True   
4457          False            False                False           True   
3653          False            False                False           True   
3654          False            False                False           True   

     is_Financials_first is_Government_first is_Industrials_first  \
3939 

In [46]:
result

,ISIN_,is_euro_mean,is_fixed_mean,Rating_mean,is_FRANCE_first,is_ITALY_first,is_GERMANY_first,is_NETHERLANDS_first,is_SPAIN_first,is_Financials_first,is_Government_first,is_Industrials_first,is_Utilities_first,B_Price_min,B_Price_max,B_Price_median,Coupon_min,Coupon_max,Coupon_median,Frequency_min,Frequency_max,MidPrice_min,MidPrice_max,MidPrice_median,MidYTM_min,MidYTM_max,MidYTM_median,SpreadvsBenchmarkMid_min,SpreadvsBenchmarkMid_max,SpreadvsBenchmarkMid_median,MidASWSpread_min,MidASWSpread_max,MidASWSpread_median,MidZSpread_min,MidZSpread_max,MidZSpread_median,GSpreadMid_min,GSpreadMid_max,GSpreadMid_median,MidModifiedDuration_min,MidModifiedDuration_max,MidModifiedDuration_median,MidConvexity_min,MidConvexity_max,MidConvexity_median,MidEffectiveDuration_min,MidEffectiveDuration_max,MidEffectiveDuration_median,MidEffectiveConvexity_min,MidEffectiveConvexity_max,MidEffectiveConvexity_median,Year_maturity_min,Month_maturity_min,Day_maturity_min,cluster
3939,XS2194370727,1.0,1.0,16.666667,False,False,False,False,True,True,False,False,False,89.0,95.0,89.5,1.125,1.125,1.125,12,12,89.339996,94.490692,90.083767,2.32647,4.23467,3.61214,127.604187,176.200348,140.128616,57.220539,84.865852,71.591511,60.088818,88.342869,75.379278,128.613968,165.996567,140.810097,3.46795,4.68134,4.142885,15.50301,27.244881,23.87388,3.11903,4.42848,3.76811,13.94323,25.358271,19.570975,2027.0,6.0,23.0,1
3590,XS1968846532,1.0,1.0,15.333333,False,False,False,False,True,True,False,False,False,91.0,93.0,92.0,1.125,1.125,1.125,12,12,91.276161,94.079071,92.899166,3.14699,4.05199,3.833165,112.821037,220.197617,132.144451,43.272888,114.193398,58.8211,44.871275,119.914936,58.53453,112.066788,227.341995,134.078812,2.19236,3.39375,2.758955,6.97037,16.371599,10.59093,2.26488,3.1817,2.56197,5.28738,14.02183,9.5865,2026.0,3.0,27.0,1
4457,XS2484587048,1.0,1.0,13.333333,False,False,False,False,True,False,False,False,False,90.0,90.0,90.0,2.592,2.592,2.592,12,12,91.209473,91.209473,91.209473,3.96444,3.96444,3.96444,137.08754,137.08754,137.08754,74.599419,74.599419,74.599419,77.241409,77.241409,77.241409,182.540356,182.540356,182.540356,6.54891,6.54891,6.54891,52.0919,52.0919,52.0919,4.298991,4.298991,4.298991,30.850787,30.850787,30.850787,2031.0,2.0,25.0,3
3653,XS2013574038,1.0,1.0,15.333333,False,False,False,False,True,True,False,False,False,89.0,95.0,90.0,1.375,1.375,1.375,12,12,89.652252,94.780617,90.671837,2.82935,4.56358,4.34748,141.264435,282.050262,238.8181,77.954376,167.89772,147.811142,80.692581,177.08741,155.55117,150.950516,283.935028,238.357193,2.58778,3.64908,3.34967,9.23744,17.62627,15.91995,2.39417,3.46687,3.02198,8.54631,16.19191,13.17485,2026.0,6.0,19.0,3
3654,XS2013745703,1.0,1.0,11.666667,False,False,False,False,True,True,False,False,False,90.0,92.0,91.0,1.0,1.0,1.0,12,12,89.916153,92.929367,92.029526,3.01281,4.05684,3.75771,102.264091,191.130508,123.59293,31.75876,91.26329,47.497066,33.06206,96.55489,47.267663,109.356148,193.712891,127.952415,2.84012,3.59702,2.9511,13.62974,17.37661,14.188945,2.6385,3.35106,2.748815,10.15737,15.43089,10.88809,2026.0,6.0,21.0,1


In [47]:
df_bond_clustering_filled[df_bond_clustering_filled['ISIN_']=='XS2236363573']

,ISIN_,is_euro_mean,is_fixed_mean,Rating_mean,is_FRANCE_first,is_ITALY_first,is_GERMANY_first,is_NETHERLANDS_first,is_SPAIN_first,is_Financials_first,is_Government_first,is_Industrials_first,is_Utilities_first,B_Price_min,B_Price_max,B_Price_median,Coupon_min,Coupon_max,Coupon_median,Frequency_min,Frequency_max,MidPrice_min,MidPrice_max,MidPrice_median,MidYTM_min,MidYTM_max,MidYTM_median,SpreadvsBenchmarkMid_min,SpreadvsBenchmarkMid_max,SpreadvsBenchmarkMid_median,MidASWSpread_min,MidASWSpread_max,MidASWSpread_median,MidZSpread_min,MidZSpread_max,MidZSpread_median,GSpreadMid_min,GSpreadMid_max,GSpreadMid_median,MidModifiedDuration_min,MidModifiedDuration_max,MidModifiedDuration_median,MidConvexity_min,MidConvexity_max,MidConvexity_median,MidEffectiveDuration_min,MidEffectiveDuration_max,MidEffectiveDuration_median,MidEffectiveConvexity_min,MidEffectiveConvexity_max,MidEffectiveConvexity_median,Year_maturity_min,Month_maturity_min,Day_maturity_min,cluster
3985,XS2236363573,0.977778,1.0,17.5,False,False,False,False,True,False,False,False,False,86.529528,94.530604,90.735407,1.875,1.875,1.875,12,12,86.84127,94.926643,91.091461,2.82396,4.4393,3.70355,106.666039,240.187836,148.157104,40.52055,110.48162,110.48162,41.70961,87.764959,87.764959,158.674089,179.161051,179.161051,2.385497,5.00435,2.385497,17.839323,28.182018,17.839323,2.228408,4.57407,2.228408,5.77398,20.946393,14.499675,2028.0,6.0,24.0,1
